Before you turn this lab in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
DRIVER = "Mike"
NAVIGATOR = "JOnathan"

# Optimization II Lab

Welcome to the optimization II lab! By the end of this lab you will have

- Performed optimization on a deep learning model with several different optimizers
- Visualized optimizer dynamics via TensorBoard
- Visualized optimizer performance via SacredBoard

Let's get started!

# Unit Test Variables

The following code defines variables that will be used in subsequent unit tests. Do not attempt to redefine any of these variables throughout the notebook!

In [2]:
from IPython.display import HTML

def passed():
    print('✅')

## Task

- Define a convolutional neural network trainer `CNNTrainer` in `trainers.py`

## Remarks

- You can either fill in the code below or directly edit `trainers.py`

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten

Using TensorFlow backend.


In [4]:
from trainer import Trainer

class CNNTrainer(Trainer):
    """Convolutional Neural Network Classifier"""

    def build_model(self):
        from keras.models import Sequential
        from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten

        model = Sequential()
        model.add(Conv2D(36, kernel_size=2, strides=1, input_shape=self.X[0].shape, activation='relu'))
        model.add(MaxPooling2D(2))
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dense(self.Y[0].shape[0], activation='softmax'))
        
        

        self.model = model
        
import trainers
trainers.CNNTrainer = CNNTrainer

# `CNNTrainer` Tests

In [5]:
import numpy as np
from trainers import CNNTrainer

cnn = CNNTrainer(config={})
cnn.load_data()
cnn.build_model()

nb_conv = len([layer for layer in cnn.model.layers if layer.name.startswith('conv')])
assert nb_conv >= 1
assert cnn.model.input_shape == (None, 28, 28, 1)
assert cnn.model.output_shape == (None, 10)
X_ = np.random.randn(16, 28, 28, 1)
y_pred = cnn.model.predict_classes(X_, verbose=0)
assert np.all(0 <= y_pred) and np.all(y_pred < 10)

passed()

✅


## Task

- Optimize your `CNNTrainer` model with every optimizer in keras with the exception of `TFOptimizer`

## Suggestion

- Start with a simple model like `MLRTrainer` and only consider a small number of training examples so you can debug quickly until you're sure everything is working correctly and them experiment with larger slower `CNNTrainer`

## Requirement

- Keep the sacred [Mongo Observer](http://sacred.readthedocs.io/en/latest/observers.html#mongo-observer) so you can view the results afterward in [sacredboard](https://github.com/chovanecm/sacredboard)

In [6]:
from train import ex
import keras
import keras.backend as K
from sacred.observers import MongoObserver
mongo_observer = MongoObserver.create()
ex.observers.append(mongo_observer)

for trainer in ['MLRTrainer']:
    run = ex.run(config_updates=dict(trainer=trainer), options={'--name': trainer})
    K.clear_session()

INFO - MLRTrainer - Running command 'main'
INFO - MLRTrainer - Started run with ID "17"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                7850      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 7,850.0
Trainable params: 7,850.0
Non-trainable params: 0.0
_________________________________________________________________
INFO:tensorflow:Summary name dense_3/kernel:0 is illegal; using dense_3/kernel_0 instead.


INFO - tensorflow - Summary name dense_3/kernel:0 is illegal; using dense_3/kernel_0 instead.


INFO:tensorflow:Summary name dense_3/bias:0 is illegal; using dense_3/bias_0 instead.


INFO - tensorflow - Summary name dense_3/bias:0 is illegal; using dense_3/bias_0 instead.


Epoch 1/10
1000/1000 [==============================] - 0s - loss: 2.1800 - acc: 0.2260     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 1.8760 - acc: 0.4890     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 1.6473 - acc: 0.6420     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 1.4741 - acc: 0.6980     

INFO - MLRTrainer - Result: 0.9530881419181824
INFO - MLRTrainer - Completed after 0:00:04


## Task

- Visualize optimizer dynamics with TensorBoard

## Requirements

- Select out plots which track only training loss during your experiment runs in TensorBoard with an appropriate regex (e.g. `loss`)
- Take a screenshot
- Load it into a `IPython.display.Image` object called `tensorboard_screenshot`
- Display it

In [7]:
optimizers = ['adam','rmsprop', 'adagrad','adadelta','adam','adamax','nadam','sgd']
for trainer in ['CNNTrainer']:
    for optimizer in optimizers:
        run = ex.run(config_updates=dict(trainer=trainer, optimizer=optimizer), options={'--name': optimizer})
        K.clear_session()

INFO - adam - Running command 'main'
INFO - adam - Started run with ID "18"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 27, 27, 36)        180       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 36)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6084)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               608500    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 609,690.0
Trainable params: 609,690.0
Non-trainable params: 0.0
_________________________________________________________________
INFO:tensorflow:Summary name conv2d_1/kernel:0 is illegal; using conv2d_1/kernel_0 instead.


INFO - tensorflow - Summary name conv2d_1/kernel:0 is illegal; using conv2d_1/kernel_0 instead.


INFO:tensorflow:Summary name conv2d_1/bias:0 is illegal; using conv2d_1/bias_0 instead.


INFO - tensorflow - Summary name conv2d_1/bias:0 is illegal; using conv2d_1/bias_0 instead.


...

INFO - tensorflow - Summary name dense_2/kernel:0 is illegal; using dense_2/kernel_0 instead.


INFO:tensorflow:Summary name dense_2/bias:0 is illegal; using dense_2/bias_0 instead.


INFO - tensorflow - Summary name dense_2/bias:0 is illegal; using dense_2/bias_0 instead.


Epoch 1/10
1000/1000 [==============================] - 0s - loss: 2.2379 - acc: 0.3010     

INFO - sgd - Result: 0.5050789251327514
INFO - sgd - Completed after 0:00:11


# TensorBoard Screenshot Test

In [8]:
tensorboard_screenshot = Image('tensorboard_screenshot.png')

NameError: name 'Image' is not defined

In [ ]:
import IPython

assert type(tensorboard_screenshot) == IPython.core.display.Image
assert 'PNG' in str(tensorboard_screenshot.data) or 'JPG' in str(tensorboard_screenshot.data)

passed()

## Task

- Visualize optimizer performance with sacredboard

## Requirements

- Sort the optimizer runs in sacredboard by loss
- Take a screenshot
- Load it into a `IPython.display.Image` object called `sacredboard_screenshot`
- Display it

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# SacredBoard Screenshot Test

In [ ]:
from IPython.display import Image

In [ ]:
sacredboard_screenshot = Image('sacredboard_screenshot.png')

In [ ]:
sacredboard_screenshot

In [ ]:
import IPython

assert type(sacredboard_screenshot) == IPython.core.display.Image
assert 'PNG' in str(sacredboard_screenshot.data) or 'JPG' in str(sacredboard_screenshot.data)

passed()

## Question

- Which optimizer did you find worked best? Does that surprise you? If so, why?

The best optimzier was nadam. Nadam is Adam RMSprop with Nesterov momentum. No that is not suprising because it normalized by the mean and variance of the previous gradients. 

## Question

- What is the intution behind that optimitzer?

Momentum helps avoid saddle points in optimization. In addition, RMSprop helps avoid exploding, and vanishing, gradients. Combined, this is the best optimizer.

# Challenge Activities

- Tune the hyperparameters of the best optimizer
- Do the same thing in TensorFlow